# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
cities = pd.read_csv(r"C:/Users/Owner/uofo-por-data-pt-03-2020-u-c/06-Python-APIs/HW/output_data/cities.csv", encoding='utf-8')
cities.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
humidity = cities["Humidity"].astype(float)
maxhum = humidity.max()
locations = cities[["Lat", "Lng"]]

In [22]:
hmap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhum,point_radius=3)
hmap.add_layer(heat_layer)
hmap


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
city_df_wcon = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
city_df_wcon = city_df_wcon.dropna(how='any')
city_df_wcon.reset_index(inplace=True)
del city_df_wcon['index']
city_df_wcon.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = []

for i in range(len(city_df_wcon)):
    lat = city_df_wcon.loc[i]['Lat']
    lng = city_df_wcon.loc[i]['Lng']
    
    params = {
        "location" : f"{lat},{lng}",
        "radius" : 5000, 
        "types" : "hotel",
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    json = requested.json()
    try: 
        hotel_df.append(json['results'][0]['name'])
    except:
        hotel_df.append("")
city_df_wcon["Hotel Name"] = hotel_list
city_df_wcon = city_df_wcon.dropna(how='any')
city_df_wcon.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,


In [34]:
#hotel_df.to_csv('hotel_export_data.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hmap.add_layer(markers)
hmap
# Display figure


Figure(layout=FigureLayout(height='420px'))